# Modeling
Esse Notebook visa apresentar a etapa de definição do modelo de machine learning usado para fazer a predição da ```ICU```.<br>
Após a etapa de *Feature Engineering* os dados estão prontos para serem usados de maneira a auxiliar a classificação.<br>

<a name="sumario"></a>
# Sumário
<!--ts-->
   * [Importe das Bibliotecas](#import)
   * [Leitura dos dados](#leitura)
   * [Metrica de avaliação](#metrics)
   * [Escalar o AGE_PERCENTIL](#age)
   * [Features com alta correlação](#corr)
<!--te-->

<a name="import"></a>
# Importe das Bibliotecas

## Bibliotecas Externas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

## Bibliotecas Internas

In [7]:
import sys
sys.path.insert(1, "../src/")
from supervised import LazyClassifier
from modeling import run_model_cv

-----------------
Retornar ao [Sumário](#sumario)

<a name="leitura"></a>
# Leitura dos dados
Faz a leitura dos dados que serão usados para a modelagem.<br>
**```df : pd.DataFrame```** é o DataFrame contendo os dados que já passaram pela etapa de *feature engineering*.

In [2]:
# Leitura do raw data desse  projeto no Github
df = pd.read_csv('../data/processed/df_featured.csv', index_col='Unnamed: 0')
df.index.names = ['PATIENT_VISIT_IDENTIFIER']

# Imprime na tela todas as colunas em vez de as 10 primeiras e 10 ultimas
pd.set_option('max_columns', df.shape[1])
df.head()

,AGE_ABOVE65,AGE_SCALED,DISEASE GROUPING 1,DISEASE GROUPING 3,DISEASE GROUPING 5,HTN,IMMUNOCOMPROMISED,ALBUMIN,BE_VENOUS,FFA,GGT,GLUCOSE,HEMOGLOBIN,LACTATE,LINFOCITOS,NEUTROPHILES,P02_VENOUS,PCR,SAT02_VENOUS,SODIUM,DIMER,BLOODPRESSURE_DIASTOLIC_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,BLOODPRESSURE_SISTOLIC_MEDIAN,OXYGEN_SATURATION_MEDIAN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_MAX_MEDIAN_RESIDUAL,HEART_RATE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_SISTOLIC_DIFF,ICU
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0.111111,0.0,0.0,0.0,1.0,0.0,0.605263,-1.000000,-0.742004,-0.958528,-0.851024,-0.024390,1.000000,-0.655602,-0.789916,-0.704142,-0.999244,0.345679,-0.142857,-0.974174,0.185185,-0.389831,-0.285714,-0.276923,0.842105,-0.435897,-0.285714,-0.179487,-0.017708,-0.671642,0.842105,-1.0,0
1,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.605263,-1.000000,-0.742004,-0.988318,-0.891993,-0.170732,1.000000,-0.582988,-0.891557,-0.704142,-0.951607,0.345679,-0.028571,-0.978029,0.185185,-0.457627,-0.107143,-0.523077,1.000000,-0.316239,-0.357143,-0.179487,-0.019761,-0.567164,1.000000,-1.0,0
2,0,0.222222,0.0,0.0,0.0,0.0,0.0,0.605263,-0.863874,-0.742004,-0.958528,-0.206704,0.073171,1.000000,-0.815353,-0.952781,-0.656805,-0.962193,0.481481,0.066667,-0.978029,0.086420,-0.525424,0.404762,-0.384615,0.684211,-0.316239,-0.428571,-0.247863,-0.018606,-0.567164,0.684211,-1.0,0
3,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.605263,-1.000000,-0.742004,-0.958528,-0.891993,-0.268293,1.000000,-0.481328,-0.921569,-0.704142,-0.981474,0.345679,0.066667,-0.991905,-0.160494,-0.525424,-0.142857,-0.692308,0.947368,-0.247863,-0.428571,-0.418803,-0.021172,-0.507463,0.947368,-1.0,0
4,1,0.777778,0.0,0.0,1.0,0.0,1.0,0.000000,-0.884817,-0.675906,-0.751168,-0.810056,-0.475610,-0.793745,-0.798755,-0.893157,-0.917160,-0.443856,-0.703704,0.066667,-0.979262,0.901235,-0.169492,0.142857,-0.338462,0.789474,-0.042735,-0.428571,0.316239,-0.018221,-0.328358,0.789474,-1.0,1


<a name="metrics"></a>
# Métrica de avaliação:
A métrica de avaliação utilizada para determinar o modelo de machine learning será o **ROC AUC**.<br>
* **ROC** do inglês *Receiver Operating Characteristics* é uma curva plotada entre os eixos *Taxa de Verdadeiros Positivos* e a *Taxa de Falsos Positivos*. Essa métrica mede o quanto o modelo consegue separar o binômio de saída entre suas duas classes (```ICU = 0``` e ```ICU = 1```).
* **AUC** do inglês *Area Under the Curve* é a maneira de resumir a curva **ROC** em um único valor real, de maneira a facilitar a comparação com outros modelos. Seria o mesmo que calcular a inegral do ROC<br>
* **ROC AUC**<br>
    * O valor será próximo de 0.5 se o modelo não tiver capacidade de separar as variáveis de saída em suas respectivas classes. Imagine que o modelo Tem 50% de verdadeiro positivo e 50% de falso positivo, ou seja, acerta em 50% das tentativas.
    * Quanto mais o valor se aproxima de 1.0, mais capacidade o modelo tem de separar os dois grupos, ou classes, acertando muito mais os Verdadeiros Positivos que errando.
    * O inverso também é verdadeiro. Quanto mais o valor se aproxima de 0, significa que o modelo também tem a capacidade de separar o binômio de saída em dois grupos, só que nesse caso, errando muito, pois teria um valor de Falsos Verdadeiros muito grande. Como a saída tem 2 estados, sendo 0 ou 1, poderia ser usado o inverso (NOT) da resposta e ainda assim o modelo acertaria bastante.

# Dummy Classifier
Essa etapa visa criar um modelo genérico que faz predições dos dados de entrada usando regras bem simples. Ter um modelo "bobo" que usa uma regra simples é importante para avaliar o modelo final, pois se espera do modelo a ser implementado que ao menos seja melhor que escolher aleatóriamente entre as alternativas de saída, ou então "chutar" a alternativa de saída mais comum.

In [3]:
run_model_cv('DummyClassifier', DummyClassifier, df, 7, 10)

NameError: name 'run_model_cv' is not defined

Vemos que o DummyClassifier encontrou um ROC AUC Médio de **0.5**, logo, essa deve ser a meta inicial a ser batida.

In [4]:
run_model_cv('RandomForestClassifier', RandomForestClassifier, df, 70, 5)

NameError: name 'run_model_cv' is not defined

In [5]:
run_model_cv('RandomForestClassifier', RandomForestClassifier, df, 5, 5)

NameError: name 'run_model_cv' is not defined

In [36]:
y = df["ICU"]
X = df.drop(["ICU"], axis=1)
model_list = []
for _ in range(100):
    print(_)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True)#test_size=0.10)
    clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
    models,predictions = clf.fit(X_train, X_test, y_train, y_test)
    model_list.append(models['ROC AUC'])

0


 97%|███████████████████████████████████████████████████████████████████████████████▏  | 28/29 [00:02<00:00, 13.64it/s]

[14:37:45] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 12.51it/s]


1


...

[14:41:36] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 10.82it/s]


99


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 11.47it/s]

[14:41:38] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [37]:
pd.DataFrame([pd.DataFrame(model_list).mean(axis=0), pd.DataFrame(model_list).std(axis=0)], index=['mean','std']).T.sort_values(by='mean', ascending=False)

,mean,std
Model,,
RandomForestClassifier,0.73,0.04
ExtraTreesClassifier,0.72,0.05
SVC,0.71,0.04
LGBMClassifier,0.71,0.05
XGBClassifier,0.70,0.04
NearestCentroid,0.70,0.05
NuSVC,0.70,0.04
RidgeClassifierCV,0.70,0.04
BernoulliNB,0.70,0.04


In [53]:
pd.DataFrame(model_list).mean().sort_values(ascending=False)

Model
RandomForestClassifier          0.72
NuSVC                           0.72
SVC                             0.72
XGBClassifier                   0.71
ExtraTreesClassifier            0.71
LGBMClassifier                  0.71
BernoulliNB                     0.71
RidgeClassifierCV               0.71
BaggingClassifier               0.70
LinearDiscriminantAnalysis      0.70
RidgeClassifier                 0.70
LinearSVC                       0.70
CalibratedClassifierCV          0.70
LogisticRegression              0.70
NearestCentroid                 0.69
AdaBoostClassifier              0.67
GaussianNB                      0.67
SGDClassifier                   0.65
Perceptron                      0.65
DecisionTreeClassifier          0.65
QuadraticDiscriminantAnalysis   0.64
ExtraTreeClassifier             0.64
KNeighborsClassifier            0.62
PassiveAggressiveClassifier     0.62
LabelSpreading                  0.56
LabelPropagation                0.56
DummyClassifier                 

In [42]:
for ml in model_list:
    print(ml.sort_index())

Model
AdaBoostClassifier              0.71
BaggingClassifier               0.64
BernoulliNB                     0.71
CalibratedClassifierCV          0.70
DecisionTreeClassifier          0.64
DummyClassifier                 0.50
ExtraTreeClassifier             0.64
ExtraTreesClassifier            0.78
GaussianNB                      0.72
KNeighborsClassifier            0.64
LGBMClassifier                  0.72
LabelPropagation                0.58
LabelSpreading                  0.58
LinearDiscriminantAnalysis      0.70
LinearSVC                       0.71
LogisticRegression              0.72
NearestCentroid                 0.77
NuSVC                           0.73
PassiveAggressiveClassifier     0.76
Perceptron                      0.73
QuadraticDiscriminantAnalysis   0.71
RandomForestClassifier          0.74
RidgeClassifier                 0.70
RidgeClassifierCV               0.71
SGDClassifier                   0.70
SVC                             0.76
XGBClassifier                   

# Referências
* https://towardsdatascience.com/modeling-teaching-a-machine-learning-algorithm-to-deliver-business-value-ad0205ca4c86
* https://docs.microsoft.com/en-us/windows/ai/windows-ml/what-is-a-machine-learning-model
* https://medium.com/@datadrivenscience/7-stages-of-machine-learning-a-framework-33d39065e2c9
* https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc
* https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5
* https://medium.com/bio-data-blog/entenda-o-que-%C3%A9-auc-e-roc-nos-modelos-de-machine-learning-8191fb4df772